# Topic Modeling the 1-Hop Monster Scrape

Build a topic model of the ~7k Wikipedia pages generated by the 1-hop spider scrape of the manually selected monster seed pages

In [1]:
spider_pages_1_dir = './spider-pages-depth-1'
train_pos_dir='./labeled/train/positive'
train_neg_dir='./labeled/train/negative'
test_pos_dir='./labeled/test/positive'
test_neg_dir='./labeled/test/negative'

To keep the token count down, we'll ignore rare words - ones that appear this many times or fewer in the corpus

In [2]:
rare_word_threshold = 50

Number of topics to model the wikipedia 1-hop data set

In [3]:
num_topics = 250

### Read the whole article text into dictionary keyed on filename

In [4]:
import os
from io import open

ext = '.txt'
file_dict = {}

txt_files = [i for i in os.listdir(spider_pages_1_dir) if os.path.splitext(i)[1] == ext]

for f in txt_files:
    with open(os.path.join(spider_pages_1_dir, f), encoding="utf-8") as file_object:
        file_dict[f] = file_object.read()
        
print ('Read ', len(file_dict), ' text files')

Read  7067  text files


In [5]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

From working with the dataset, I've learned that we should have some data-set-specific stopwords...

In [6]:
wikipedia_stop_words = {
    
    "retrieved",  # the references on each page often have a retrieval data as part of the listing
    "edit",       # the text of the label of the edit button often appears inline in the article text
    "isbn",       # appears frequenly in references
    "archived",   # the phrase 'archived from the original' appears frequently in the references
    "original",
    "url"
}

In [7]:
import re
import string

def tokenize(text):
    lda_tokens = []
    text = re.sub("http\S*", "URL", text)
    tokens = word_tokenize(text)
    for token in tokens:
        if not token.isalpha():
            continue
        #if token in string.punctuation:
        #    continue
        #if token in ["'", '"', '`', "''"]:
        #    continue
        if len(token) < 4:
            continue
        lc = token.lower()
        if lc in stop_words:
            continue
        if lc in wikipedia_stop_words:
            continue
        #if lc == 'url':
        #    continue
        lda_tokens.append(lc)
    return lda_tokens


In [8]:
# Tokenize the corpus
tokenized_docs = []
tokenized_docs_filenames = []

for f in file_dict:
    tokenized_docs_filenames.append(f)
    tokens = tokenize(file_dict[f])
    tokenized_docs.append(tokens)

In [9]:
# Peek into the tokenized data as a sanity check
print(tokenized_docs_filenames[0])
print(tokenized_docs[0])

Samuel Marinus Zwemer.txt
['american', 'missionary', 'traveler', 'scholar', 'samuel', 'marinus', 'zwemer', 'born', 'april', 'vriesland', 'michigan', 'united', 'states', 'died', 'april', 'aged', 'york', 'city', 'united', 'states', 'education', 'hope', 'college', 'brunswick', 'theological', 'seminary', 'spouse', 'elizabeth', 'wilkes', 'samuel', 'marinus', 'zwemer', 'april', 'april', 'nicknamed', 'apostle', 'islam', 'american', 'missionary', 'traveler', 'scholar', 'born', 'vriesland', 'michigan', 'received', 'hope', 'college', 'holland', 'received', 'brunswick', 'theological', 'seminary', 'brunswick', 'degrees', 'include', 'hope', 'college', 'muskingum', 'college', 'rutgers', 'college', 'ordained', 'reformed', 'church', 'ministry', 'pella', 'iowa', 'classis', 'missionary', 'busrah', 'bahrein', 'locations', 'arabia', 'member', 'arabian', 'mission', 'founder', 'american', 'mission', 'hospital', 'bahrain', 'zwemer', 'served', 'egypt', 'also', 'traveled', 'widely', 'asia', 'minor', 'elected',

In [10]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [11]:
word_freqs = dict()
for tokenized_doc in tokenized_docs:
    for word in tokenized_doc:
        if word in word_freqs:
            word_freqs[word] += 1
        else:
            word_freqs[word] = 1
            

Let's take a look at the word list prior to filtering out rare words...

In [12]:
sorted_word_freqs = sortFreqDict(word_freqs)
print('number of words: ', len(word_freqs))
sorted_word_freqs

number of words:  405820


[(71687, 'film'),
 (59745, 'also'),
 (42839, 'help'),
 (39967, 'first'),
 (37351, 'world'),
 (35949, 'target'),
 (35063, 'error'),
 (30437, 'history'),
 (29737, 'university'),
 (29457, 'time'),
 (29453, 'march'),
 (29264, 'press'),
 (29011, 'october'),
 (28610, 'series'),
 (28427, 'films'),
 (28251, 'january'),
 (27863, 'april'),
 (27665, 'july'),
 (26894, 'december'),
 (26869, 'june'),
 (26781, 'september'),
 (26325, 'august'),
 (25523, 'november'),
 (25157, 'february'),
 (24811, 'york'),
 (24415, 'united'),
 (24082, 'game'),
 (23598, 'john'),
 (23137, 'states'),
 (22515, 'used'),
 (22458, 'american'),
 (22193, 'many'),
 (21816, 'city'),
 (21509, 'would'),
 (21217, 'king'),
 (20037, 'book'),
 (19548, 'science'),
 (19362, 'godzilla'),
 (19005, 'times'),
 (18992, 'early'),
 (18866, 'years'),
 (18704, 'life'),
 (18564, 'century'),
 (18211, 'list'),
 (18122, 'article'),
 (18036, 'people'),
 (17272, 'story'),
 (17014, 'later'),
 (16792, 'news'),
 (16672, 'national'),
 (16640, 'main'),
 (16

Filter our rare words 

In [13]:
highly_filtered_tokenized_docs = []
filtered_words = set()
for tokenized_doc in tokenized_docs:
    tokens = []
    for word in tokenized_doc:
        if word_freqs[word] > rare_word_threshold:
            tokens.append(word)
            filtered_words.add(word)
    highly_filtered_tokenized_docs.append(tokens)

In [14]:
print('Vocabulary size after rare word removal:')
print(len(filtered_words))

Vocabulary size after rare word removal:
26583


In [15]:
# Peek into the tokenized data as a sanity check
print(tokenized_docs_filenames[0])
print(highly_filtered_tokenized_docs[0])

Samuel Marinus Zwemer.txt
['american', 'missionary', 'traveler', 'scholar', 'samuel', 'marinus', 'zwemer', 'born', 'april', 'michigan', 'united', 'states', 'died', 'april', 'aged', 'york', 'city', 'united', 'states', 'education', 'hope', 'college', 'brunswick', 'theological', 'seminary', 'spouse', 'elizabeth', 'wilkes', 'samuel', 'marinus', 'zwemer', 'april', 'april', 'nicknamed', 'apostle', 'islam', 'american', 'missionary', 'traveler', 'scholar', 'born', 'michigan', 'received', 'hope', 'college', 'holland', 'received', 'brunswick', 'theological', 'seminary', 'brunswick', 'degrees', 'include', 'hope', 'college', 'college', 'rutgers', 'college', 'ordained', 'reformed', 'church', 'ministry', 'pella', 'iowa', 'missionary', 'locations', 'arabia', 'member', 'arabian', 'mission', 'founder', 'american', 'mission', 'hospital', 'bahrain', 'zwemer', 'served', 'egypt', 'also', 'traveled', 'widely', 'asia', 'minor', 'elected', 'fellow', 'royal', 'geographical', 'society', 'london', 'appointed', '

In [16]:
from gensim import corpora

# Create Dictionary
id2word = corpora.Dictionary(highly_filtered_tokenized_docs)
# Create Corpus
texts = highly_filtered_tokenized_docs
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

C:\Users\peter\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 2), (12, 2), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 4), (19, 1), (20, 1), (21, 1), (22, 1), (23, 8), (24, 4), (25, 2), (26, 7), (27, 2), (28, 3), (29, 3)]


In [17]:
import gensim.models.ldamodel
from pathlib import Path
from gensim.test.utils import datapath

filename = "./lda-model"

lda_file = Path(filename)
if not lda_file.is_file():
    print("Building model")
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations=1000,
                                       alpha=.1,
                                       eta=.1,
                                       update_every=0,
                                       passes=12)
    lda_model.save(filename)
else:
    print("Loading pre-built model from ", filename)
    lda_model = gensim.models.ldamodel.LdaModel.load(filename)


Loading pre-built model from  ./lda-model


In [18]:
from pprint import pprint
pprint(lda_model.print_topics(num_topics))

[(0,
  '0.036*"cross" + 0.017*"temple" + 0.010*"memes" + 0.010*"temples" + '
  '0.008*"columns" + 0.005*"meme" + 0.005*"used" + 0.005*"also" + '
  '0.004*"force" + 0.004*"century"'),
 (1,
  '0.069*"virginia" + 0.014*"israel" + 0.011*"kingdom" + 0.010*"richmond" + '
  '0.009*"judah" + 0.008*"little" + 0.006*"house" + 0.005*"chesapeake" + '
  '0.004*"northern" + 0.004*"march"'),
 (2,
  '0.014*"detective" + 0.011*"steampunk" + 0.010*"malaysia" + '
  '0.010*"jewellery" + 0.008*"fiction" + 0.007*"mystery" + 0.007*"stories" + '
  '0.006*"zone" + 0.006*"gold" + 0.006*"twilight"'),
 (3,
  '0.045*"norse" + 0.019*"edda" + 0.016*"saga" + 0.015*"cologne" + '
  '0.012*"elves" + 0.010*"snorri" + 0.009*"mythology" + 0.008*"sagas" + '
  '0.007*"prose" + 0.007*"poetic"'),
 (4,
  '0.225*"best" + 0.066*"awards" + 0.051*"award" + 0.047*"year" + '
  '0.017*"movie" + 0.014*"golden" + 0.014*"trailer" + 0.013*"poster" + '
  '0.011*"winners" + 0.010*"spot"'),
 (5,
  '0.014*"science" + 0.009*"scientific" + 0.00

### Look at the top topics for each document as a sanity check

In [19]:
for filename, text in zip(tokenized_docs_filenames, highly_filtered_tokenized_docs):
    bow_doc = id2word.doc2bow(text)
    doc_to_topic = lda_model.get_document_topics(bow = bow_doc, minimum_probability = 0.05)
    print(filename)
    print(doc_to_topic)
    print()

Samuel Marinus Zwemer.txt
[(80, 0.6232163), (95, 0.339967)]

P-51 Dragon Fighter.txt
[(24, 0.10588387), (35, 0.22130865), (133, 0.055055667), (168, 0.15108147), (173, 0.33093238)]

Wired News.txt
[(60, 0.06901993), (61, 0.28659058), (113, 0.06089103), (203, 0.52486503)]

Voyage to the Prehistoric Planet.txt
[(11, 0.053142507), (173, 0.765843), (186, 0.06655265)]

Succubi in fiction.txt
[(22, 0.13468148), (157, 0.17143306), (169, 0.06414659), (193, 0.15128891)]

Woolworth Building.txt
[(174, 0.6710343), (176, 0.20722085), (218, 0.057340115)]

Susumu Kurobe.txt
[(77, 0.3990855), (88, 0.1837519), (169, 0.11706006), (238, 0.117064655)]

Motif-Index of Folk-Literature.txt
[(5, 0.07552465), (26, 0.38567877), (146, 0.10263426), (157, 0.1183052), (206, 0.14352149), (239, 0.117486775)]

Terminator 2-COLON- Judgment Day.txt
[(32, 0.5421219), (38, 0.31270164)]

Playmobil-COLON- The Movie.txt
[(38, 0.07148438), (116, 0.8477647)]

Musicarello.txt
[(122, 0.69152087), (191, 0.21471652)]

NES.txt
[(60

Umberto Eco.txt
[(35, 0.07903835), (65, 0.22441661), (98, 0.21699856), (226, 0.15146314)]

Star Wars-COLON- The Force Awakens.txt
[(38, 0.16791849), (130, 0.500486), (220, 0.3263997)]

Northern (genre).txt
[(97, 0.05889945), (122, 0.17705616), (146, 0.0782095), (191, 0.32171157), (202, 0.12669894), (237, 0.14381832)]

Soul.txt
[(199, 0.99556714)]

Spectrophilia.txt
[(82, 0.79259706), (223, 0.06015044)]

Shotacon.txt
[(19, 0.50835115), (122, 0.34038273), (191, 0.055407826)]

Sandman.txt
[(17, 0.29534188), (105, 0.051636677), (111, 0.053018313), (137, 0.08329053), (157, 0.20272918), (169, 0.060527567), (173, 0.08969742)]

Ángel Sala.txt
[(38, 0.15379141), (47, 0.37210736), (150, 0.05001998), (191, 0.15289561), (213, 0.08717143)]

Systema Naturae.txt
[(104, 0.055334978), (117, 0.48645887), (232, 0.43456635)]

Nangoku no hada.txt
[(123, 0.5705539), (173, 0.11336358), (237, 0.0694678), (238, 0.097108975)]

Prototype (1992 film).txt
[(21, 0.79332954), (92, 0.11113758)]

Religion.txt
[(192, 0

Vörðr.txt
[(3, 0.06993439), (59, 0.057173252), (82, 0.053124115), (130, 0.067203596), (157, 0.50141394), (199, 0.07049148)]

Peripheral vision.txt
[(5, 0.079473466), (29, 0.27218932), (84, 0.19149229), (195, 0.081366226), (205, 0.14952081)]

Walt Disney Television (production company).txt
[(21, 0.084674224), (45, 0.8333336)]

Wulfings.txt
[(3, 0.5246554), (103, 0.242965), (121, 0.12159586)]

Monsters (2010 film).txt
[(38, 0.7496754), (88, 0.08657693), (173, 0.07896732)]

Times Square.txt
[(22, 0.57994735), (174, 0.093755685), (176, 0.20705941)]

Polish Film School.txt
[(122, 0.7416928), (191, 0.16085956)]

Turok-COLON- Son of Stone (film).txt
[(21, 0.22863406), (35, 0.05402566), (67, 0.07968633), (105, 0.06612409), (173, 0.38324207)]

WolfCop.txt
[(173, 0.68783444), (221, 0.099696085)]

Sephora.txt
[(46, 0.12799226), (60, 0.07544503), (69, 0.41853645), (134, 0.05113207), (176, 0.19780114)]

Prima Games.txt
[(60, 0.118096575), (110, 0.28233802), (113, 0.14307474), (126, 0.08859358), (20

OED.txt
[(206, 0.95038784)]

Taper Jean Girl.txt
[(175, 0.96692044)]

Waterloo Station.txt
[(64, 0.37768942), (80, 0.28694758), (176, 0.15230568), (218, 0.054476477), (230, 0.051859416)]

Succubous.txt
[(29, 0.16420846), (59, 0.064638294), (66, 0.1227748), (84, 0.06912401), (140, 0.11224065), (234, 0.09470029)]

Template talk-COLON-Kimiyoshi Yasuda.txt
[(126, 0.19637004)]

Republic of China.txt
[(44, 0.18248183), (52, 0.21816365), (56, 0.13181195), (74, 0.3460321), (118, 0.05625126)]

Wendigo.txt
[(65, 0.06502914), (137, 0.2640729), (157, 0.14776303), (218, 0.07022334), (237, 0.058203854), (239, 0.06485107)]

On Translating Beowulf.txt
[(59, 0.066488035), (121, 0.8500486)]

Monotheism.txt
[(95, 0.25812557), (139, 0.22324212), (178, 0.08244909), (192, 0.14022996), (199, 0.20514652)]

Regarding Henry.txt
[(35, 0.18984638), (38, 0.13443068), (39, 0.09186698), (173, 0.3833837)]

Space opera.txt
[(97, 0.6800191), (172, 0.15807468), (220, 0.07514093)]

Solar variation.txt
[(25, 0.9267744), (

Night at the Museum-COLON- Secret of the Tomb.txt
[(38, 0.9641465)]

Tremors (1990 film).txt
[(173, 0.4810078), (219, 0.4258898)]

Tarzan (2013 film).txt
[(111, 0.4579254), (173, 0.34853354)]

Turksploitation.txt
[(122, 0.665286), (191, 0.18575142)]

Xing Tian.txt
[(44, 0.58005416), (98, 0.06735306), (103, 0.07209743), (157, 0.16353333)]

Tech noir.txt
[(97, 0.3039273), (122, 0.42065695), (191, 0.17436363)]

Salafism.txt
[(74, 0.29568437), (95, 0.08063132), (161, 0.39408), (192, 0.18503596)]

Playmobil.txt
[(29, 0.08220945), (55, 0.17460872), (60, 0.1904706), (116, 0.19504844)]

Zolpidem.txt
[(43, 0.19000666), (137, 0.33053857), (195, 0.38931918)]

Perception.txt
[(5, 0.2817513), (29, 0.124408014), (205, 0.46422258)]

Stanford University Press.txt
[(61, 0.081572995), (65, 0.16868544), (125, 0.4919148), (214, 0.11623377)]

The Leviathan (2015 film).txt
[]

Spy film.txt
[(122, 0.23446406), (191, 0.4876668), (202, 0.18841502)]

Venus.txt
[(25, 0.089917324), (186, 0.765303), (231, 0.062647

...

Mesopotamia.txt
[(95, 0.17087765), (155, 0.39311293), (236, 0.10872683)]

Minamoto no Mitsunaka.txt
[(44, 0.08518109), (47, 0.18943615), (95, 0.05297844), (103, 0.08191046), (135, 0.062184684), (217, 0.24091554), (238, 0.10283924)]

Locus Award.txt
[(4, 0.1535785), (97, 0.79347223)]

Lantern Festival.txt
[(44, 0.39636663), (157, 0.14532952), (203, 0.3850208)]

List of fantasy authors.txt
[(35, 0.12947394), (97, 0.46293408), (102, 0.36374962)]

Mušḫuššu.txt
[(103, 0.83704793)]

Nebula Award.txt
[(4, 0.12495064), (97, 0.8104885)]

Lindworm.txt
[(103, 0.072462544), (157, 0.12742023), (168, 0.1075004), (225, 0.48474488)]

Magic ring.txt
[(9, 0.055654038), (16, 0.255642), (97, 0.10055911), (103, 0.051579345), (157, 0.112192124), (239, 0.26700073)]

Near East.txt
[(52, 0.45904887), (236, 0.07874977), (237, 0.27521306)]

Officer of arms.txt
[(74, 0.095210105), (225, 0.8127312)]

Khordeh Avesta.txt
[(155, 0.9359767)]

List of dragons in popular culture.txt
[(168, 0.26677626), (193, 0.68465257)

Extinction.txt
[(5, 0.08183829), (89, 0.06856987), (104, 0.34644854), (149, 0.1508957), (151, 0.07837798), (181, 0.061789133), (245, 0.07142216)]

Frank Zappa.txt
[(35, 0.08060739), (111, 0.07263241), (142, 0.80989224)]

Air raid siren.txt
[(194, 0.9822893)]

-SLASH-Film.txt
[(4, 0.16896577), (35, 0.10528928), (61, 0.062100958), (86, 0.112623245), (113, 0.06389611), (134, 0.17537636), (169, 0.100994706)]

Ape Canyon.txt
[(23, 0.058907155), (64, 0.23185647), (82, 0.05171443), (157, 0.05465847), (173, 0.12865774), (192, 0.1650528), (237, 0.101961344), (244, 0.11780005)]

Benny Andersson.txt
[(35, 0.111232005), (111, 0.8408674)]

Alien Resurrection.txt
[(38, 0.11387445), (160, 0.2521453), (221, 0.59732527)]

Doug Stewart (game designer).txt
[(6, 0.12906475)]

Category-COLON-CS1 maint-COLON- unfit URL.txt
[(14, 0.836905)]

Arethusa (mythology).txt
[(98, 0.18520257), (185, 0.15161547), (207, 0.25185144), (216, 0.31322804)]

A Nightmare on Elm Street (franchise).txt
[(102, 0.80050015), (173,


European eel.txt
[(12, 0.08935046), (52, 0.13825071), (104, 0.07706826), (149, 0.43513307), (234, 0.069280125)]

Aliens vs. Predator-COLON- Requiem.txt
[(38, 0.1967137), (160, 0.16394955), (173, 0.18087439), (183, 0.27829012), (221, 0.1451549)]

Elements of the Cthulhu Mythos.txt
[(137, 0.5087457), (157, 0.13350496), (244, 0.22098815)]

Ahead of Their Time.txt
[(120, 0.07138842), (142, 0.86727697)]

Category-COLON-Redirects from moves.txt
[(14, 0.8346518)]

Futurama.txt
[(86, 0.060235456), (108, 0.14176065), (132, 0.058076136), (169, 0.33499476), (170, 0.28826073)]

Fresno, California.txt
[(230, 0.9643556)]

Dan O'Bannon.txt
[(35, 0.54886204), (51, 0.061981432), (145, 0.20256561)]

Eleusinian Mysteries.txt
[(40, 0.053344302), (101, 0.77186894), (139, 0.16244693)]

Aurora Plastics Corporation.txt
[(21, 0.055925887), (29, 0.17976995), (36, 0.33799374), (60, 0.24068692)]

Blu-ray.txt
[(60, 0.15679216), (96, 0.8397261)]

Etsushi Toyokawa.txt
[(35, 0.051506158), (145, 0.42817566), (173, 0.

Fantastic Four.txt
[(105, 0.9050701)]

Euterpe.txt
[(66, 0.4194045), (98, 0.1393256), (101, 0.05870115), (112, 0.2207717)]

Aliens-COLON- The Computer Game (1986 video game).txt
[(110, 0.14158833), (160, 0.7819663)]

Cultural icon.txt
[(65, 0.9361929)]

Corin Hardy.txt
[(21, 0.3129928), (35, 0.17735551), (38, 0.14038548), (173, 0.09454954), (227, 0.058686428)]

Datura.txt
[(43, 0.16758467), (117, 0.07914258), (137, 0.051421683), (149, 0.18425617), (153, 0.30591115), (195, 0.054144092), (239, 0.06382139)]

Anthology-COLON- Movie Themes 1974–1998.txt
[(120, 0.9046276)]

Diffuse reflection.txt
[(29, 0.6126559), (124, 0.35471082)]

Gary Gygax.txt
[(6, 0.07764065), (69, 0.66959625), (81, 0.1796104), (83, 0.0657289)]

Androktasiai.txt
[(66, 0.69639254), (185, 0.18655859)]

Arthur Rankin Jr..txt
[(35, 0.17446245), (51, 0.054922275), (102, 0.6188642), (108, 0.056492265)]

Athens.txt
[(65, 0.052676328), (118, 0.11615293), (230, 0.3431921), (236, 0.37784192)]

Against the Giants.txt
[(81, 0.2386

Dog.txt
[(149, 0.10343615), (151, 0.71042997), (195, 0.051402006)]

Fouke, Arkansas.txt
[(136, 0.0518401), (162, 0.11813875), (184, 0.07762223), (230, 0.5613568), (237, 0.08915041)]

Escape from L.A..txt
[(102, 0.059764545), (120, 0.22890976), (173, 0.4995333)]

Ann Darrow.txt
[(51, 0.8246879), (88, 0.11168229), (110, 0.05199492)]

Beastwars (album).txt
[(21, 0.055955477), (111, 0.21567033), (120, 0.25185552), (143, 0.051677536), (175, 0.051339675)]

Azathoth (short story).txt
[(244, 0.9352149)]

Category-COLON-Articles with unsourced statements from September 2016.txt
[(14, 0.8490849)]

Aliens-COLON- Colonial Marines Technical Manual.txt
[(160, 0.8893945)]

Eternity.txt
[(5, 0.088028096), (90, 0.10761427), (139, 0.09451406), (199, 0.49096832)]

9-1-1.txt
[(60, 0.24811302), (74, 0.29509196), (134, 0.058298223), (159, 0.26059875)]

Eastern gorilla.txt
[(12, 0.77806836), (149, 0.11994465)]

Asian cinema.txt
[(20, 0.13431202), (52, 0.35133073), (191, 0.2794064)]

Foam rubber.txt
[(29, 0.7

### What topics have high weights in the manually-labeled monster documents?

In [39]:
def get_topics_for_text(text, threshold):
    topics = set()
    all_tokens = tokenize(text)
    tokens = []
    for word in all_tokens:
        if word_freqs[word] > rare_word_threshold:
            tokens.append(word)
    bow_doc = id2word.doc2bow(tokens)
    doc_to_topic = lda_model.get_document_topics(bow = bow_doc, minimum_probability = threshold)
    return doc_to_topic

In [40]:
from os import listdir
from os.path import isfile, join

monster_documents = [join(train_pos_dir,f) for f in listdir(train_pos_dir) if isfile(join(train_pos_dir, f))]
more_monster_documents = [join(test_pos_dir,f) for f in listdir(test_pos_dir) if isfile(join(test_pos_dir, f))]

monster_documents.extend(more_monster_documents)

monster_topics = set()

for f in monster_documents:
    with open(f, encoding="utf-8") as file_object:
        text = file_object.read()
        doc_to_topic = get_topics_for_text(text, 0.05)
        print(f)
        print(doc_to_topic)
        for topic in doc_to_topic:
            print(topic[0], ' ---- ', lda_model.print_topic(topic[0]))
            monster_topics.add(topic[0])
        print()

print('total number of monster topics (so far): ', len(monster_topics))

./labeled/train/positive\Dragon.txt
[(97, 0.07325695), (168, 0.49136847), (218, 0.10297265), (225, 0.067408524)]
97  ----  0.036*"fantasy" + 0.034*"fiction" + 0.019*"science" + 0.009*"world" + 0.007*"series" + 0.007*"stories" + 0.006*"magic" + 0.006*"literature" + 0.005*"history" + 0.005*"works"
168  ----  0.119*"dragon" + 0.032*"dragons" + 0.009*"european" + 0.007*"help" + 0.007*"sigurd" + 0.006*"serpent" + 0.005*"error" + 0.005*"target" + 0.005*"also" + 0.004*"mythology"
218  ----  0.141*"target" + 0.138*"error" + 0.132*"help" + 0.019*"harvnb" + 0.005*"sfnm" + 0.005*"press" + 0.004*"university" + 0.003*"york" + 0.003*"also" + 0.002*"early"
225  ----  0.041*"arms" + 0.021*"heraldry" + 0.011*"heraldic" + 0.011*"coat" + 0.008*"also" + 0.007*"used" + 0.005*"black" + 0.005*"flag" + 0.005*"white" + 0.005*"shield"

./labeled/train/positive\Vampire.txt
[(100, 0.055113595), (157, 0.12010571), (201, 0.6147919), (239, 0.050385024)]
100  ----  0.014*"paranormal" + 0.010*"scientific" + 0.009*"sci

./labeled/train/positive\Godzilla.txt
[(51, 0.063701585), (88, 0.45474136), (123, 0.36543062)]
51  ----  0.076*"kong" + 0.052*"king" + 0.015*"film" + 0.013*"island" + 0.008*"skull" + 0.008*"films" + 0.006*"cooper" + 0.006*"zombie" + 0.006*"conan" + 0.005*"dead"
88  ----  0.108*"godzilla" + 0.019*"monsters" + 0.017*"king" + 0.015*"monster" + 0.012*"film" + 0.011*"mothra" + 0.010*"ghidorah" + 0.009*"toho" + 0.009*"mechagodzilla" + 0.007*"japan"
123  ----  0.036*"godzilla" + 0.029*"film" + 0.013*"japanese" + 0.013*"king" + 0.012*"toho" + 0.012*"films" + 0.010*"galbraith" + 0.009*"help" + 0.009*"ryfle" + 0.009*"monster"

./labeled/train/positive\Them!_(1954_film).txt
[(20, 0.065476105), (67, 0.06669602), (173, 0.6412305)]
20  ----  0.044*"film" + 0.020*"films" + 0.007*"movie" + 0.007*"movies" + 0.006*"first" + 0.005*"made" + 0.005*"also" + 0.005*"cinema" + 0.005*"motion" + 0.004*"feature"
67  ----  0.024*"stone" + 0.015*"rolling" + 0.008*"plane" + 0.007*"fiends" + 0.006*"kingston" + 0.006*

./labeled/train/positive\Goblin.txt
[(42, 0.20373516), (97, 0.3581739), (157, 0.29153678)]
42  ----  0.033*"snake" + 0.030*"snakes" + 0.011*"fairy" + 0.010*"fairies" + 0.009*"lilith" + 0.008*"venom" + 0.007*"water" + 0.006*"serpent" + 0.004*"kelpie" + 0.004*"also"
97  ----  0.036*"fantasy" + 0.034*"fiction" + 0.019*"science" + 0.009*"world" + 0.007*"series" + 0.007*"stories" + 0.006*"magic" + 0.006*"literature" + 0.005*"history" + 0.005*"works"
157  ----  0.011*"also" + 0.010*"mythology" + 0.010*"folklore" + 0.007*"death" + 0.006*"creatures" + 0.005*"creature" + 0.005*"often" + 0.004*"people" + 0.004*"legend" + 0.004*"known"

./labeled/train/positive\Harpy.txt
[(40, 0.3891952), (101, 0.2340671), (112, 0.063375786), (225, 0.19545531)]
40  ----  0.013*"greek" + 0.013*"deities" + 0.006*"cave" + 0.006*"games" + 0.005*"mysteries" + 0.005*"mythological" + 0.005*"mythology" + 0.004*"hermes" + 0.004*"ancient" + 0.003*"ploutonion"
101  ----  0.017*"greek" + 0.009*"zeus" + 0.005*"mythology" + 0.

./labeled/train/positive\Cloverfield.txt
[(38, 0.45952788), (39, 0.050764874), (116, 0.08462592), (169, 0.13737799), (173, 0.091453195)]
38  ----  0.038*"film" + 0.011*"million" + 0.010*"july" + 0.009*"office" + 0.008*"march" + 0.008*"april" + 0.007*"december" + 0.007*"october" + 0.007*"january" + 0.007*"november"
39  ----  0.039*"trek" + 0.036*"star" + 0.017*"abrams" + 0.012*"november" + 0.010*"gaiman" + 0.010*"film" + 0.007*"darkness" + 0.006*"neil" + 0.006*"series" + 0.006*"july"
116  ----  0.019*"film" + 0.009*"runner" + 0.009*"movie" + 0.009*"transformers" + 0.008*"blade" + 0.007*"december" + 0.006*"july" + 0.006*"martin" + 0.005*"april" + 0.005*"october"
169  ----  0.027*"season" + 0.026*"series" + 0.021*"episode" + 0.016*"show" + 0.013*"episodes" + 0.010*"television" + 0.008*"september" + 0.008*"october" + 0.006*"february" + 0.006*"march"
173  ----  0.030*"film" + 0.007*"films" + 0.006*"movie" + 0.005*"release" + 0.005*"directed" + 0.005*"time" + 0.005*"released" + 0.005*"horror

90  ----  0.022*"monster" + 0.020*"time" + 0.016*"doctor" + 0.013*"high" + 0.010*"november" + 0.008*"cybermen" + 0.007*"doll" + 0.006*"back" + 0.006*"dolls" + 0.006*"travel"

./labeled/train/positive\Succubus.txt
[(22, 0.056505464), (100, 0.09617669), (103, 0.050532848), (137, 0.080644675), (157, 0.2308906), (239, 0.1849465)]
22  ----  0.038*"park" + 0.024*"central" + 0.024*"york" + 0.020*"times" + 0.016*"street" + 0.016*"square" + 0.015*"avenue" + 0.012*"city" + 0.012*"circle" + 0.011*"october"
100  ----  0.014*"paranormal" + 0.010*"scientific" + 0.009*"science" + 0.008*"psychic" + 0.008*"research" + 0.007*"parapsychology" + 0.006*"skeptical" + 0.006*"phenomena" + 0.005*"pseudoscience" + 0.005*"evidence"
103  ----  0.010*"also" + 0.010*"king" + 0.005*"name" + 0.005*"form" + 0.005*"text" + 0.004*"translation" + 0.004*"gilgamesh" + 0.004*"demons" + 0.004*"marduk" + 0.004*"gods"
137  ----  0.030*"sleep" + 0.012*"entity" + 0.009*"great" + 0.008*"paralysis" + 0.008*"insomnia" + 0.007*"appe

168  ----  0.119*"dragon" + 0.032*"dragons" + 0.009*"european" + 0.007*"help" + 0.007*"sigurd" + 0.006*"serpent" + 0.005*"error" + 0.005*"target" + 0.005*"also" + 0.004*"mythology"
185  ----  0.017*"hesiod" + 0.012*"press" + 0.012*"university" + 0.011*"theogony" + 0.009*"zeus" + 0.009*"west" + 0.009*"library" + 0.008*"greek" + 0.008*"apollodorus" + 0.008*"harvard"
236  ----  0.026*"greek" + 0.022*"ancient" + 0.010*"greece" + 0.010*"athens" + 0.008*"century" + 0.008*"period" + 0.007*"history" + 0.006*"empire" + 0.006*"roman" + 0.005*"city"

./labeled/test/positive\Kraken.txt
[(59, 0.05564555), (104, 0.136167), (106, 0.06745992), (157, 0.30450037), (234, 0.054255363)]
59  ----  0.025*"english" + 0.023*"language" + 0.017*"german" + 0.015*"languages" + 0.012*"germanic" + 0.007*"dialects" + 0.007*"modern" + 0.007*"used" + 0.007*"words" + 0.007*"dutch"
104  ----  0.010*"loch" + 0.010*"species" + 0.009*"ness" + 0.008*"monster" + 0.006*"animals" + 0.006*"evolution" + 0.005*"pmid" + 0.005*"larg

### Remove topics from the set that have high weight in the negative-labeled documents

In [41]:
non_monster_documents = [join(train_neg_dir,f) for f in listdir(train_neg_dir) if isfile(join(train_neg_dir, f))]
non_more_monster_documents = [join(test_neg_dir,f) for f in listdir(test_neg_dir) if isfile(join(test_neg_dir, f))]

non_monster_documents.extend(non_more_monster_documents)

non_monster_topics = set()

for f in non_monster_documents:
    with open(f, encoding="utf-8") as file_object:
        text = file_object.read()
        doc_to_topic = get_topics_for_text(text, 0.05)

        print(f)
        print(doc_to_topic)
        for topic in doc_to_topic:
            print(topic[0], ' ---- ', lda_model.print_topic(topic[0]))
            non_monster_topics.add(topic[0])
        print()

print('total number of non monster topics: ', len(non_monster_topics))

./labeled/train/negative\Clothing (1).txt
[(13, 0.874561), (29, 0.080023184)]
13  ----  0.005*"hypnosis" + 0.005*"also" + 0.004*"clothing" + 0.004*"american" + 0.003*"tattoos" + 0.003*"people" + 0.003*"press" + 0.003*"many" + 0.003*"body" + 0.003*"ufos"
29  ----  0.015*"used" + 0.008*"also" + 0.006*"water" + 0.005*"process" + 0.005*"steel" + 0.004*"system" + 0.004*"sound" + 0.004*"surface" + 0.004*"light" + 0.004*"material"

./labeled/train/negative\Flag of the Qing dynasty (1).txt
[(0, 0.09998488), (44, 0.67345035), (133, 0.10109782), (225, 0.06533236)]
0  ----  0.036*"cross" + 0.017*"temple" + 0.010*"memes" + 0.010*"temples" + 0.008*"columns" + 0.005*"meme" + 0.005*"used" + 0.005*"also" + 0.004*"force" + 0.004*"century"
44  ----  0.052*"chinese" + 0.026*"china" + 0.015*"dynasty" + 0.011*"emperor" + 0.007*"qing" + 0.006*"also" + 0.006*"history" + 0.005*"characters" + 0.005*"people" + 0.005*"traditional"
133  ----  0.016*"states" + 0.015*"united" + 0.014*"force" + 0.014*"navy" + 0.012*

./labeled/train/negative\Caveman.txt
[(0, 0.0708508), (9, 0.2691171), (97, 0.057435054), (151, 0.15274672), (153, 0.07076283), (157, 0.111763544), (173, 0.118378446)]
0  ----  0.036*"cross" + 0.017*"temple" + 0.010*"memes" + 0.010*"temples" + 0.008*"columns" + 0.005*"meme" + 0.005*"used" + 0.005*"also" + 0.004*"force" + 0.004*"century"
9  ----  0.035*"hercules" + 0.018*"heart" + 0.018*"heracles" + 0.014*"hero" + 0.014*"tolkien" + 0.011*"rings" + 0.010*"komodo" + 0.009*"lord" + 0.005*"blood" + 0.005*"hydra"
97  ----  0.036*"fantasy" + 0.034*"fiction" + 0.019*"science" + 0.009*"world" + 0.007*"series" + 0.007*"stories" + 0.006*"magic" + 0.006*"literature" + 0.005*"history" + 0.005*"works"
151  ----  0.017*"human" + 0.013*"humans" + 0.011*"pmid" + 0.009*"homo" + 0.008*"evolution" + 0.008*"years" + 0.007*"species" + 0.007*"dogs" + 0.007*"modern" + 0.006*"wolf"
153  ----  0.024*"dinosaurs" + 0.016*"dinosaur" + 0.009*"birds" + 0.006*"issn" + 0.006*"extinction" + 0.005*"early" + 0.005*"pmid" 

149  ----  0.015*"species" + 0.008*"shark" + 0.007*"pmid" + 0.006*"food" + 0.005*"ants" + 0.005*"sharks" + 0.005*"animal" + 0.005*"prey" + 0.005*"also" + 0.004*"white"
151  ----  0.017*"human" + 0.013*"humans" + 0.011*"pmid" + 0.009*"homo" + 0.008*"evolution" + 0.008*"years" + 0.007*"species" + 0.007*"dogs" + 0.007*"modern" + 0.006*"wolf"
199  ----  0.033*"soul" + 0.012*"body" + 0.009*"life" + 0.007*"souls" + 0.007*"spirit" + 0.007*"philosophy" + 0.007*"mind" + 0.007*"death" + 0.007*"human" + 0.005*"spiritual"

./labeled/train/negative\Komodo dragon.txt
[(9, 0.2684334), (52, 0.057448093), (104, 0.09078998), (149, 0.35415196), (168, 0.08798499), (234, 0.054744065)]
9  ----  0.035*"hercules" + 0.018*"heart" + 0.018*"heracles" + 0.014*"hero" + 0.014*"tolkien" + 0.011*"rings" + 0.010*"komodo" + 0.009*"lord" + 0.005*"blood" + 0.005*"hydra"
52  ----  0.009*"world" + 0.008*"islands" + 0.007*"south" + 0.006*"population" + 0.006*"asia" + 0.006*"east" + 0.006*"north" + 0.005*"island" + 0.005*"co

86  ----  0.028*"channel" + 0.020*"network" + 0.018*"news" + 0.018*"television" + 0.013*"radio" + 0.008*"august" + 0.008*"programming" + 0.007*"media" + 0.007*"networks" + 0.007*"cable"
205  ----  0.020*"mirror" + 0.016*"perception" + 0.014*"opossum" + 0.010*"neurons" + 0.008*"visual" + 0.008*"brain" + 0.007*"pmid" + 0.006*"cortex" + 0.005*"system" + 0.005*"tibetan"

./labeled/train/negative\Florida.txt
[(177, 0.32041138), (237, 0.56072056)]
177  ----  0.042*"florida" + 0.019*"aztec" + 0.018*"mexico" + 0.014*"spanish" + 0.010*"hasbro" + 0.007*"aztecs" + 0.006*"conquest" + 0.006*"commodus" + 0.006*"empire" + 0.006*"tenochtitlan"
237  ----  0.017*"state" + 0.010*"river" + 0.008*"states" + 0.008*"area" + 0.008*"north" + 0.007*"lake" + 0.006*"national" + 0.006*"population" + 0.006*"south" + 0.006*"west"

./labeled/train/negative\Geomagnetic.txt
[(25, 0.81325907), (29, 0.10752253)]
25  ----  0.015*"solar" + 0.012*"tyrannosaurus" + 0.010*"earth" + 0.008*"field" + 0.008*"magnetic" + 0.006*"pm

95  ----  0.015*"church" + 0.009*"christian" + 0.009*"bible" + 0.007*"hebrew" + 0.006*"jewish" + 0.006*"book" + 0.006*"catholic" + 0.006*"holy" + 0.005*"christianity" + 0.005*"john"
111  ----  0.027*"album" + 0.019*"music" + 0.017*"band" + 0.013*"tarzan" + 0.013*"song" + 0.012*"songs" + 0.012*"time" + 0.009*"released" + 0.009*"rock" + 0.008*"land"
125  ----  0.038*"university" + 0.013*"college" + 0.011*"texas" + 0.009*"school" + 0.007*"research" + 0.006*"students" + 0.006*"hall" + 0.006*"campus" + 0.006*"state" + 0.005*"education"
218  ----  0.141*"target" + 0.138*"error" + 0.132*"help" + 0.019*"harvnb" + 0.005*"sfnm" + 0.005*"press" + 0.004*"university" + 0.003*"york" + 0.003*"also" + 0.002*"early"
224  ----  0.016*"angola" + 0.012*"vietnamese" + 0.011*"bengali" + 0.011*"atlantic" + 0.009*"rock" + 0.006*"cappella" + 0.005*"help" + 0.005*"english" + 0.005*"language" + 0.005*"target"
233  ----  0.032*"bass" + 0.028*"music" + 0.015*"double" + 0.009*"string" + 0.009*"instruments" + 0.008*

...

20  ----  0.044*"film" + 0.020*"films" + 0.007*"movie" + 0.007*"movies" + 0.006*"first" + 0.005*"made" + 0.005*"also" + 0.005*"cinema" + 0.005*"motion" + 0.004*"feature"
60  ----  0.010*"digital" + 0.009*"music" + 0.007*"audio" + 0.006*"media" + 0.006*"also" + 0.006*"company" + 0.006*"used" + 0.006*"format" + 0.006*"computer" + 0.005*"tape"
96  ----  0.035*"disc" + 0.017*"jordan" + 0.015*"video" + 0.013*"laser" + 0.012*"discs" + 0.009*"format" + 0.008*"players" + 0.008*"optical" + 0.007*"player" + 0.006*"sony"

./labeled/train/negative\Metaphor.txt
[(5, 0.4821504), (54, 0.22570686), (63, 0.08217875), (98, 0.09701021)]
5  ----  0.014*"science" + 0.009*"scientific" + 0.007*"theory" + 0.005*"example" + 0.005*"also" + 0.004*"social" + 0.004*"research" + 0.004*"used" + 0.003*"psychology" + 0.003*"philosophy"
54  ----  0.034*"gothic" + 0.009*"story" + 0.008*"novel" + 0.008*"character" + 0.008*"fiction" + 0.006*"also" + 0.005*"matrix" + 0.005*"literature" + 0.005*"film" + 0.004*"characters"
6

./labeled/train/negative\Idaho.txt
[(198, 0.14791045), (237, 0.763996)]
198  ----  0.028*"team" + 0.025*"idaho" + 0.020*"super" + 0.020*"game" + 0.019*"bowl" + 0.018*"league" + 0.013*"sports" + 0.012*"february" + 0.011*"football" + 0.010*"january"
237  ----  0.017*"state" + 0.010*"river" + 0.008*"states" + 0.008*"area" + 0.008*"north" + 0.007*"lake" + 0.006*"national" + 0.006*"population" + 0.006*"south" + 0.006*"west"

./labeled/train/negative\Lion.txt
[(104, 0.49975142), (149, 0.11796894), (151, 0.16916618)]
104  ----  0.010*"loch" + 0.010*"species" + 0.009*"ness" + 0.008*"monster" + 0.006*"animals" + 0.006*"evolution" + 0.005*"pmid" + 0.005*"large" + 0.005*"water" + 0.005*"mammals"
149  ----  0.015*"species" + 0.008*"shark" + 0.007*"pmid" + 0.006*"food" + 0.005*"ants" + 0.005*"sharks" + 0.005*"animal" + 0.005*"prey" + 0.005*"also" + 0.004*"white"
151  ----  0.017*"human" + 0.013*"humans" + 0.011*"pmid" + 0.009*"homo" + 0.008*"evolution" + 0.008*"years" + 0.007*"species" + 0.007*"dog

./labeled/train/negative\Wolf.txt
[(149, 0.20878488), (151, 0.52987826), (215, 0.054950483), (218, 0.066338755)]
149  ----  0.015*"species" + 0.008*"shark" + 0.007*"pmid" + 0.006*"food" + 0.005*"ants" + 0.005*"sharks" + 0.005*"animal" + 0.005*"prey" + 0.005*"also" + 0.004*"white"
151  ----  0.017*"human" + 0.013*"humans" + 0.011*"pmid" + 0.009*"homo" + 0.008*"evolution" + 0.008*"years" + 0.007*"species" + 0.007*"dogs" + 0.007*"modern" + 0.006*"wolf"
215  ----  0.057*"bear" + 0.045*"bears" + 0.021*"black" + 0.016*"american" + 0.014*"grizzly" + 0.009*"genome" + 0.009*"brown" + 0.007*"human" + 0.007*"goodall" + 0.006*"ursus"
218  ----  0.141*"target" + 0.138*"error" + 0.132*"help" + 0.019*"harvnb" + 0.005*"sfnm" + 0.005*"press" + 0.004*"university" + 0.003*"york" + 0.003*"also" + 0.002*"early"

./labeled/train/negative\World War II.txt
[(11, 0.7499227), (24, 0.08089215), (52, 0.052488837), (218, 0.06828479)]
11  ----  0.015*"soviet" + 0.014*"help" + 0.013*"target" + 0.013*"error" + 0.007*

52  ----  0.009*"world" + 0.008*"islands" + 0.007*"south" + 0.006*"population" + 0.006*"asia" + 0.006*"east" + 0.006*"north" + 0.005*"island" + 0.005*"country" + 0.004*"history"
104  ----  0.010*"loch" + 0.010*"species" + 0.009*"ness" + 0.008*"monster" + 0.006*"animals" + 0.006*"evolution" + 0.005*"pmid" + 0.005*"large" + 0.005*"water" + 0.005*"mammals"
147  ----  0.016*"hakluyt" + 0.014*"fangoria" + 0.013*"iguana" + 0.011*"effects" + 0.007*"stereoscopic" + 0.006*"pinus" + 0.006*"clam" + 0.006*"gwangi" + 0.006*"poltergeist" + 0.005*"special"
149  ----  0.015*"species" + 0.008*"shark" + 0.007*"pmid" + 0.006*"food" + 0.005*"ants" + 0.005*"sharks" + 0.005*"animal" + 0.005*"prey" + 0.005*"also" + 0.004*"white"
234  ----  0.013*"species" + 0.008*"cells" + 0.007*"pmid" + 0.007*"biology" + 0.006*"blood" + 0.006*"squid" + 0.005*"also" + 0.005*"lung" + 0.005*"cell" + 0.005*"body"
237  ----  0.017*"state" + 0.010*"river" + 0.008*"states" + 0.008*"area" + 0.008*"north" + 0.007*"lake" + 0.006*"nat

./labeled/test/negative\Gorilla.txt
[(12, 0.77211857), (149, 0.12376417)]
12  ----  0.013*"gorilla" + 0.012*"apes" + 0.012*"life" + 0.010*"orangutan" + 0.008*"chimpanzees" + 0.008*"human" + 0.008*"chimpanzee" + 0.007*"great" + 0.007*"orangutans" + 0.006*"jung"
149  ----  0.015*"species" + 0.008*"shark" + 0.007*"pmid" + 0.006*"food" + 0.005*"ants" + 0.005*"sharks" + 0.005*"animal" + 0.005*"prey" + 0.005*"also" + 0.004*"white"

./labeled/test/negative\Japan.txt
[(21, 0.071993224), (52, 0.25500092), (74, 0.1957737), (118, 0.15108566), (238, 0.18754937)]
21  ----  0.028*"parameter" + 0.028*"maint" + 0.027*"discouraged" + 0.027*"link" + 0.007*"film" + 0.006*"movie" + 0.006*"series" + 0.005*"voice" + 0.004*"jerry" + 0.004*"lake"
52  ----  0.009*"world" + 0.008*"islands" + 0.007*"south" + 0.006*"population" + 0.006*"asia" + 0.006*"east" + 0.006*"north" + 0.005*"island" + 0.005*"country" + 0.004*"history"
74  ----  0.010*"united" + 0.007*"women" + 0.007*"march" + 0.006*"september" + 0.006*"gov

### Manually add topics to automatically-generated list of monster topics

In [56]:
# First, remove the non-monster topics from the monster topics
print("Original set of monster topics")
print(monster_topics)
print()

print("Topics from non-monster documents:")
print(non_monster_topics)
print()

print("Intersection of monster and non-monster topics")
intersection = monster_topics.intersection(non_monster_topics)
print(intersection)
for intersection_topic in intersection:
    print(intersection_topic, ' --- ', lda_model.print_topic(intersection_topic))

print("Remaining monster topics:")
refined_monster_topics = monster_topics.difference(non_monster_topics)
print(refined_monster_topics)

Original set of monster topics
{0, 6, 7, 8, 12, 15, 20, 21, 22, 37, 38, 39, 40, 42, 47, 51, 57, 59, 64, 67, 70, 73, 80, 82, 83, 85, 88, 89, 90, 92, 94, 95, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 110, 111, 112, 115, 116, 117, 120, 121, 122, 123, 125, 126, 137, 138, 139, 141, 146, 151, 152, 157, 158, 160, 163, 168, 169, 170, 173, 174, 185, 186, 191, 193, 198, 199, 201, 206, 213, 215, 218, 221, 225, 234, 235, 236, 237, 238, 239, 244, 249}

Topics from non-monster documents:
{0, 2, 5, 9, 10, 11, 12, 13, 20, 21, 23, 24, 25, 26, 28, 29, 33, 35, 38, 41, 44, 45, 52, 54, 56, 59, 60, 61, 62, 63, 64, 65, 66, 69, 70, 74, 80, 84, 86, 89, 91, 93, 95, 96, 97, 98, 102, 104, 110, 111, 112, 114, 117, 118, 120, 125, 126, 130, 132, 133, 134, 140, 147, 149, 151, 152, 153, 155, 157, 159, 162, 165, 166, 168, 169, 170, 172, 173, 174, 175, 176, 177, 179, 180, 181, 184, 186, 187, 190, 194, 195, 196, 198, 199, 201, 202, 203, 205, 206, 207, 208, 211, 212, 214, 215, 218, 220, 222, 224, 225, 226, 229, 230,

120  ---  0.051*"carpenter" + 0.038*"halloween" + 0.028*"john" + 0.015*"themes" + 0.012*"escape" + 0.011*"lost" + 0.009*"film" + 0.009*"soundtrack" + 0.009*"music" + 0.009*"mars"
125  ---  0.038*"university" + 0.013*"college" + 0.011*"texas" + 0.009*"school" + 0.007*"research" + 0.006*"students" + 0.006*"hall" + 0.006*"campus" + 0.006*"state" + 0.005*"education"
126  ---  0.032*"article" + 0.023*"template" + 0.019*"talk" + 0.016*"alien" + 0.015*"page" + 0.013*"please" + 0.013*"wikiproject" + 0.009*"would" + 0.008*"articles" + 0.008*"project"
Remaining monster topics:
{6, 7, 8, 137, 138, 139, 141, 15, 146, 22, 158, 160, 163, 37, 39, 40, 42, 47, 51, 57, 185, 191, 193, 67, 73, 82, 83, 85, 213, 88, 90, 92, 221, 94, 100, 101, 249, 103, 105, 106, 107, 235, 239, 115, 116, 244, 121, 122, 123}


In [50]:
manual_monster_topics = set([
3,
6,
7,
8,
9,
15,
18,
26,
27,
30,
32,
34,
37,
40,
42,
43,
51,
53,
57,
64,
66,
67,
75,
79,
81,
82,
83,
88,
89,
90,
92,
94,
97,
100,
101,
102,
103,
104,
105,
106,
109,
115,
121,
122,
123,
128,
136,
137,
138,
139,
141,
143,
147,
154,
157,
160,
168,
171,
172,
173,
183,
193,
201,
211,
221,
228,
235,
239,
242,
244]
)

In [53]:
intersection_manual_refined = manual_monster_topics.intersection(refined_monster_topics)
print(intersection_manual_refined)

manual_not_in_refined = manual_monster_topics.difference(refined_monster_topics)
print(manual_not_in_refined)
refined_not_in_manual = refined_monster_topics.difference(manual_monster_topics)
print(refined_not_in_manual)

{6, 7, 8, 137, 138, 139, 141, 15, 160, 37, 40, 42, 51, 57, 193, 67, 82, 83, 88, 90, 92, 221, 94, 100, 101, 103, 105, 106, 235, 239, 115, 244, 121, 122, 123}
{128, 3, 136, 9, 143, 18, 147, 154, 26, 27, 157, 30, 32, 34, 168, 43, 171, 172, 173, 53, 183, 64, 66, 201, 75, 79, 81, 211, 89, 97, 228, 102, 104, 109, 242}
{163, 249, 39, 73, 107, 47, 146, 116, 85, 22, 213, 185, 158, 191}


In [46]:
num_monster_articles = 0
for filename, text in zip(tokenized_docs_filenames, highly_filtered_tokenized_docs):
    bow_doc = id2word.doc2bow(text)
    doc_to_topic = lda_model.get_document_topics(bow = bow_doc, minimum_probability = 0.15)
    for topic in doc_to_topic:
        if topic[0] in refined_monster_topics:
            print(filename, ' - (', topic[0], ':', topic[1], ') ')
            print(lda_model.print_topic(topic[0]))
            print()
            num_monster_articles += 1

print (num_monster_articles, ' articles related to monster topics')


0  articles related to monster topics
